# Vector Space Model
- Rel(D, Q) = Document와 Query의 관련성
- Rel = Similarity
    - vector 상에서는 거리, 각도 로 유사도(similarity)를 구할 수 있다.
    - 유사도가 높으면 관련성이 높은 것으로 판단하여, 검색 결과를 만들 수 있다.


## How to measure similarity?
1. Euclidean distance
2. Cosine Similarity

**오늘의 학습 목표**: Euclidean distance와 Cosine similarity로 Vector space model에서 검색하기.

---

### Linked-list 만들기

- 예제 collection으로 간단하게 구현하기
- 간단하게 해보기 위해서 Posting은 python list를 이용한다.
    - Posting은 저장공간을 분리시켜야하지만, 이번 실습에서는 편의를 위해서 메모리상에 둔다.
    - list의 index를 pointer로 활용한다.

In [3]:
from collections import defaultdict

_Collections = [
    ["A", "A", "A",  "A", "A"],
    ["A", "A", "A",  "A", "B"],
    ["A", "A", "A",  "A", "C"],
]

_Vocabulary = list()
_Lexicon = defaultdict(lambda: -1)
_Document = defaultdict(int)
_Posting = list()

for d in _Collections:
    _localPosting = defaultdict(int)
    for t in d:
        if t not in _Vocabulary:
            _Vocabulary.append(t)
        _localPosting[t] += 1
    docID = len(_Document)
    _Document[docID] = max(_localPosting.values())
    for t, f in _localPosting.items():
        ptr = _Lexicon[t]
        nextPtr = len(_Posting)
        _Posting.append((docID, f, ptr))
        _Lexicon[t] = nextPtr

### TF-IDF 계산식 정의

Vector Space Model에서는 concept에 따라서 단어를 벡터 공간에 표현한다.  
concept는 controled words와 같은 뜻이다.  
단어 1개당 차원 1개를 의미한다.  
concept를 잘 표현하기 위해서 TF-IDF 기법을 이용해서 가중치(weight)로 산출한다.

In [4]:
from math import log

tf1 = lambda t: 1
tf2 = lambda _struct: _struct[1]
tf3 = lambda t:0
tf4 = lambda _struct, t: log(1 + _struct[1])
tf6 = lambda tf, maxtf, a:a+(1-a)*(tf/maxtf)
tf5 = lambda tf, maxtf: tf6(tf, maxtf, 0.5)
idf1 = lambda df, N:log(N/df) # 일반적인 idf
idf2 = lambda df, N:log(N/(1+df)) 
    # the, a 등 불용어는 모든 doc에서 나올 수 있으므로 log1=0이 될 수 있다.
    # 해결: smoothing 기법(1을 더해줌)
idf3 = lambda df, N:log((1 + N-df)/df) 
    # N-df가 0이 되는 것을 방지하기 위해서 1을 더함

### 가중치 매기기
- indexer

In [5]:
# Euclidean distance

# 두 점 간의 차이에 대한 제곱을 계속 summation할 것이다.
# 나중에 sqrt(제곱근)하면 유클리드 거리가 된다.

distance = lambda x1, x2: (x2 - x1)**2

In [6]:
N = len(_Collections)
_Weight = list()
_WLexicon = defaultdict(lambda:{"Posting":None, "DF":0})
_DocLength = defaultdict(float)
for t, ptr in _Lexicon.items():
    dfPtr = ptr
    df = 0
    while dfPtr != -1:
        _struct = _Posting[dfPtr]
        df += 1
        dfPtr = _struct[-1]
    
    wptr = len(_Weight)
    while ptr != -1:
        _struct = _Posting[ptr]
        tf = _struct[1]
        maxtf = _Document[_struct[0]]
        w = tf6(tf, maxtf, 0)* idf2(df, N)
        print("단어:{0}, 문서:{1}, 빈도:{2}, 최고빈도:{3}, 가중치={4:.2f}"
              .format(
                  t, _struct[0], _struct[1],
                  maxtf, w))
        
        ptr = _struct[-1]
        
        wStruct = (_struct[0], w)
        _Weight.append(wStruct)
        _DocLength[_struct[0]] += distance(0, w) # Cosine similarity 계산용
    _WLexicon[t]["Posting"] = wptr
    _WLexicon[t]["DF"] = df

단어:A, 문서:2, 빈도:4, 최고빈도:4, 가중치=-0.29
단어:A, 문서:1, 빈도:4, 최고빈도:4, 가중치=-0.29
단어:A, 문서:0, 빈도:5, 최고빈도:5, 가중치=-0.29
단어:B, 문서:1, 빈도:1, 최고빈도:4, 가중치=0.10
단어:C, 문서:2, 빈도:1, 최고빈도:4, 가중치=0.10


### Query 검색하기 (Euclidean distance 이용)
- query parsing

In [27]:
from math import sqrt

Q = "B B B"
result = defaultdict(float)
QueryRepr = defaultdict(int) # localPosting가 같은 역할; freq를 센다.
QueryWeight = defaultdict(float)
for t in Q.split():
    if t in _Vocabulary:  # _Vocabulary에 있는 단어만 쿼리로 사용
        QueryRepr[t] += 1
maxQuery = max(QueryRepr.values())
for t, f in QueryRepr.items():
    w = tf6(f, maxQuery, 0)*idf2(_WLexicon[t]["DF"], N)
    QueryWeight[t] = w

for t in _Vocabulary:
    ptr = _WLexicon[t]["Posting"]
    df = _WLexicon[t]["DF"]
    for _struct in _Weight[ptr:ptr+df]:
        _struct[0] # => 문서
        _struct[1] # => 가중치
        result[_struct[0]] += distance(QueryWeight[t],
                                      _struct[1])
    
result = {d:sqrt(dist)  for d, dist in result.items()}

In [28]:
result

{2: 0.3050181911435358, 1: 0.41861327484332994, 0: 0.2876820724517809}

> "B B B"라는 query에 대하여 B가 들어있는 두번째 문서가 가장 높은 유사도를 보인다.

### Query 검색하기 (Cosine similarity 이용)
- query parsing

In [29]:
Q = "A A A A B"
result = defaultdict(float)
QueryRepr = defaultdict(int)
QueryWeight = defaultdict(float)
QueryLength = 0.0
for t in Q.split():
    if t in _Vocabulary:
        QueryRepr[t] += 1
maxQuery = max(QueryRepr.values())
for t, f in QueryRepr.items():
    w = tf6(f, maxQuery, 0)*idf2(_WLexicon[t]["DF"], N)
    QueryWeight[t]= w
    QueryLength += distance(0,w)

for t in QueryRepr:
    ptr = _WLexicon[t]["Posting"]
    df = _WLexicon[t]["DF"]
    for _struct in _Weight[ptr:ptr+df]:
        _struct[0] # => 문서
        _struct[1] # => 가중치
        
        # result를 계산할 때 cosine similarity 수식을 이용한다.
        result[_struct[0]] += QueryWeight[t] * _struct[1]    
result = {d:ip/(sqrt(QueryLength)*sqrt(_DocLength[d]))
          for d, ip in result.items()}

In [30]:
_DocLength

defaultdict(float,
            {2: 0.09303609692847455,
             1: 0.09303609692847455,
             0: 0.08276097481015171})

In [31]:
result

{2: 0.889557682904279, 1: 1.0000000000000002, 0: 0.9431636564797644}

> Query와 두번째 document가 동일하기 때문에 유사도가 1로 나왔다.

---

# Posting을 local에 저장하는 방법으로 구현하기
- 위와 동일한 방법이나, Positing을 local에 저장하여 linked-list를 좀 더 원칙적으로 구현하는 실습.
- python open의 tell과 seek 메서드를 pointer로 활용함.
- 위의 실습에서 코드가 바뀐 부분은 주석처리하여 차이를 알아보기 쉽게 하였음.

### Posting File

In [14]:
from struct import pack, unpack

_Collections = [
    ["A", "A", "A",  "A", "A"],
    ["A", "A", "A",  "A", "B"],
    ["A", "A", "A",  "A", "C"],
]

_Vocabulary = list()
_Lexicon = defaultdict(lambda: -1)
_Document = defaultdict(int)
# _Posting = list()

fp = open("posting.dat", "wb")

for d in _Collections:
    _localPosting = defaultdict(int)
    for t in d:
        if t not in _Vocabulary:
            _Vocabulary.append(t)
        _localPosting[t] += 1
    docID = len(_Document)
    _Document[docID] = max(_localPosting.values())
    for t, f in _localPosting.items():
        ptr = _Lexicon[t]
#         nextPtr = len(_Posting)
#         _Posting.append((docID, f, ptr))
        nextPtr = fp.tell()
        fp.write(pack("iii", docID, f, ptr))
        _Lexicon[t] = nextPtr
        
fp.close()

### 가중치 매기기

In [15]:
N = len(_Collections)
_Weight = list()
_WLexicon = defaultdict(lambda:{"Posting":None, "DF":0})
_DocLength = defaultdict(float)

fp = open("posting.dat", "rb")
wp = open("weight.dat", "wb")

for t, ptr in _Lexicon.items():
    dfPtr = ptr
    df = 0
    while dfPtr != -1:
#         _struct = _Posting[dfPtr]
        fp.seek(dfPtr)
        _struct = unpack("iii", fp.read(4*3))
        df += 1
        dfPtr = _struct[-1]
        
#     wptr = len(_Weight)
    wptr = wp.tell()
    while ptr != -1:
#         _struct = _Posting[ptr]
        fp.seek(ptr)
        _struct = unpack("iii", fp.read(4*3))
        tf = _struct[1]
        maxtf = _Document[_struct[0]]
        w = tf6(tf, maxtf, 0)* idf2(df, N)
        print("단어:{0}, 문서:{1}, 빈도:{2}, 최고빈도:{3}, \
        가중치={4:.2f}"
              .format(
                  t, _struct[0], _struct[1],
                  maxtf, w))
        ptr = _struct[-1]
        
#         wStruct = (_struct[0], w)
#         _Weight.append(wStruct)
        wp.write(pack("if", _struct[0], w))
        _DocLength[_struct[0]] += distance(0, w)
    _WLexicon[t]["Posting"] = wptr
    _WLexicon[t]["DF"] = df
    
fp.close()
wp.close()

단어:A, 문서:2, 빈도:4, 최고빈도:4,         가중치=-0.29
단어:A, 문서:1, 빈도:4, 최고빈도:4,         가중치=-0.29
단어:A, 문서:0, 빈도:5, 최고빈도:5,         가중치=-0.29
단어:B, 문서:1, 빈도:1, 최고빈도:4,         가중치=0.10
단어:C, 문서:2, 빈도:1, 최고빈도:4,         가중치=0.10


### Query 검색하기 (Euclidean distance 이용)

In [ ]:
from math import sqrt

Q = "B B B"
result = defaultdict(float)
QueryRepr = defaultdict(int) 
QueryWeight = defaultdict(float)
for t in Q.split():
    if t in _Vocabulary:  
        QueryRepr[t] += 1
maxQuery = max(QueryRepr.values())
for t, f in QueryRepr.items():
    w = tf6(f, maxQuery, 0)*idf2(_WLexicon[t]["DF"], N)
    QueryWeight[t] = w

wp = open("weight.dat", "rb")
    
for t in _Vocabulary:
    ptr = _WLexicon[t]["Posting"]
    df = _WLexicon[t]["DF"]
#     for _struct in _Weight[ptr:ptr+df]:
    wp.seek(ptr)
    for i in range(df):
        _struct = unpack("if", wp.read(4*2))
        result[_struct[0]] += distance(QueryWeight[t],
                                      _struct[1])

wp.close()
result = {d:sqrt(dist)  for d, dist in result.items()}

In [ ]:
result

### Query 검색하기 (Cosine similarity 이용)

In [16]:
from math import sqrt
Q = "A A A A B"
result = defaultdict(float)
QueryRepr = defaultdict(int)
QueryWeight = defaultdict(float)
QueryLength = 0.0
for t in Q.split():
    if t in _Vocabulary:
        QueryRepr[t] += 1
maxQuery = max(QueryRepr.values())
fp = open("weight.dat", "rb")
for t, f in QueryRepr.items():
    w = tf6(f, maxQuery, 0)*idf2(_WLexicon[t]["DF"], N)
    QueryWeight[t]= w
    QueryLength += distance(0,w)

for t in QueryRepr:
    ptr = _WLexicon[t]["Posting"]
    df = _WLexicon[t]["DF"]
    fp.seek(ptr)
    for i in range(df):
        _struct = unpack("if", fp.read(4*2))
        _struct[0] # => 문서
        _struct[1] # => 가중치
        result[_struct[0]] += QueryWeight[t] * _struct[1]  
result = {d:ip/(sqrt(QueryLength)*sqrt(_DocLength[d]))
          for d, ip in result.items()}

fp.close()

In [17]:
result

{2: 0.8895577255065847, 1: 1.0000000393683177, 0: 0.9431637016493439}

---

# 한글 corpus로 실전 연습

In [18]:
from konlpy.corpus import kobill
from konlpy.tag import Kkma
from struct import pack, unpack


Lexicon = defaultdict(lambda:{"ptr":-1, "df":0})
# 단어:{위치, 총몇개의문서}

Documents = defaultdict(lambda:{
    "length":0.0, "ttf":0, "max":0})
# 문서:(문서벡터의길이, 총몇개의단어, 이문서에서가장많이나온단어의빈도)
# ttf = total term frequency

DocumentsList = list()
# 인덱스:문서의제목

kkma = Kkma()
# 형태소분석기(꼬꼬마)

fp = open("inverted.dat", "wb")
# TDM(frequency)

for docName in kobill.fileids():
    # Local
    document = kobill.open(docName).read() # 개별문서
    localPostings = defaultdict(int) # 각 문서의 TDM(Vector)
    # 문서정보 저장
    DocID = len(DocumentsList) # 문서의 제목 -> 숫자
    DocumentsList.append(docName) # 해당 숫자(위치)  제목 저장
    # 각 문서에서 색인어 추출 방식 (형태소분석기, 형태소의길이로정규화)
    for term in [_ for _ in kkma.morphs(document)
              if 1 < len(_) < 6]:
        localPostings[term] += 1
        # 문서 벡터 생성 (열 단어:빈도)
    # 문서의 통계정보 저장 => for weight
    Documents[DocID]["ttf"] = sum(localPostings.values())
    Documents[DocID]["max"] = max(localPostings.values())
    # Global
    for term, freq in localPostings.items():
            if term not in Lexicon:
        ptr = Lexicon[term]["ptr"]
        # 1. 단어가 첫 등장: 위치 ptr=-1
        # 2. 단어가 있을 때, ptr= 마지막 저장 위치
        postingPtr= fp.tell()
        # 파일의 어느 위치에 저장하는 지 
        fp.write(pack("iii", DocID, freq, ptr))
        # 구조체를 저장(int, int, int)
        Lexicon[term]["ptr"] = postingPtr
        # 1.1 위치 변경 : 파일의 위치
        Lexicon[term]["df"] += 1
        # for IDF
        
fp.close()

C:\Users\JINHYO\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [19]:
Documents

defaultdict(<function __main__.<lambda>()>,
            {0: {'length': 0.0, 'ttf': 883, 'max': 123},
             1: {'length': 0.0, 'ttf': 883, 'max': 119},
             2: {'length': 0.0, 'ttf': 1039, 'max': 200},
             3: {'length': 0.0, 'ttf': 893, 'max': 92},
             4: {'length': 0.0, 'ttf': 213, 'max': 11},
             5: {'length': 0.0, 'ttf': 346, 'max': 35},
             6: {'length': 0.0, 'ttf': 1803, 'max': 122},
             7: {'length': 0.0, 'ttf': 672, 'max': 35},
             8: {'length': 0.0, 'ttf': 642, 'max': 31},
             9: {'length': 0.0, 'ttf': 1855, 'max': 430}})

In [20]:
# Indexer => Weighting(weight.dat)

from math import log2

fp1 = open("inverted.dat", "rb")
fp2 = open("weight.dat", "wb")

N = len(DocumentsList)
for term, termStruct in Lexicon.items():
    ptr = termStruct["ptr"]
    wPtr = fp2.tell()
    while ptr != -1:
        fp1.seek(ptr)
        _struct = unpack("iii", fp1.read(4*3)) # 12byte
        _struct[0] # => 문서 ID
        _struct[1] # => 해당 문서에서의 빈도(tf)
        TF = _struct[1] / Documents[_struct[0]]["max"]
        IDF = log2(N/termStruct["df"])
        Documents[_struct[0]]["length"] += TF*IDF
        fp2.write(pack("if", _struct[0], TF*IDF))
        ptr = _struct[-1]
    Lexicon[term]["ptr"] = wPtr
fp2.close()
fp1.close()

In [21]:
# Query Parser => QyeryRepr(with Weight)

Query = "현행법은 입법예고와 행정예고를 통하여 정책 결정 과정에"

localPostings = defaultdict(int)

for term in [_ for _ in kkma.morphs(Query)
                if 1 < len(_) < 6]:
    localPostings[term] += 1
maxTF = max(localPostings.values())
# TTF = sum(localPostings.values()) => 나중에 BM25에서 쓸거다.
for term, freq, in localPostings.items():
    TF = freq/maxTF
    IDF = log2(N/Lexicon[term]["df"])
    localPostings[term] = TF*IDF
    
# ==> Query Representation

In [22]:
localPostings

defaultdict(int,
            {'현행법': 0.5,
             '입법': 1.660964047443681,
             '예고': 3.321928094887362,
             '행정': 0.3684827970831031,
             '통하': 1.160964047443681,
             '정책': 0.2572865864148791,
             '결정': 1.660964047443681,
             '과정': 0.6609640474436812})

# 정리

# 1. Euclidean distance
- 식:
$$
dist(q, d) = \sqrt{\sum_{t\in V}{[tf(t,q) \cdot idf(t) - tf(t,d) \cdot idf(t)]^2}}
$$

- 해석: (시그마 t ㅌ V) sqrt((D벡터t번째값-Q벡터t번째값)\*\*2)
- 문제: summation을 할때, vocabulary에 속하는 모든 t(단어)에 대해서 실행하는데 t는 query에 없는 단어일수도 있다. 이때 penalty가 발생한다.



In [32]:
searchResult = defaultdict(float)
fp = open("weight.dat", "rb")
for term, termStruct in Lexicon.items():
    ptr = termStruct["ptr"]
    fp.seek(ptr)
    for _ in range(termStruct["df"]):
        _struct = unpack("if", fp.read(4*2))
        qw = localPostings[term] # 이름은 나중에 바꾸시구여
        dw = _struct[1]
        searchResult[_struct[0]] += (qw-dw)**2
fp.close()

searchResult = {d:sqrt(dist)
               for d, dist in searchResult.items()}

for d, dist in {DocumentsList[_[0]]:_[1]
                for _ in sorted(searchResult.items(), 
                        key=lambda r:r[1])}.items():
    print({d:dist})
    print({d:{"거리":dist, "단어수": Documents[d]["ttf"]}})
    print("\n".join(kobill.open(d).read().splitlines()[:3]))
    print()
    

{'1809892.txt': 0.9785681821197749}
{'1809892.txt': {'거리': 0.9785681821197749, '단어수': 0}}
교육공무원법 일부개정법률안

(정의화의원 대표발의 )

{'1809890.txt': 1.1213888599844724}
{'1809890.txt': {'거리': 1.1213888599844724, '단어수': 0}}
지방공무원법 일부개정법률안

(정의화의원 대표발의 )

{'1809891.txt': 1.1454561206043383}
{'1809891.txt': {'거리': 1.1454561206043383, '단어수': 0}}
국가공무원법 일부개정법률안

(정의화의원 대표발의 )

{'1809893.txt': 1.3363030928544017}
{'1809893.txt': {'거리': 1.3363030928544017, '단어수': 0}}
남녀고용평등과 일 ·가정 양립 지원에 관한 법률 

일부개정법률안

{'1809899.txt': 1.5382427458729653}
{'1809899.txt': {'거리': 1.5382427458729653, '단어수': 0}}
결혼중개업의 관리에 관한 법률 일부개정법률안

(한선교의원 대표발의 )

{'1809895.txt': 2.3306777198717277}
{'1809895.txt': {'거리': 2.3306777198717277, '단어수': 0}}
하도급거래 공정화에 관한 법률 일부개정법률안

(유선호의원 대표발의 )

{'1809896.txt': 3.0498591928696674}
{'1809896.txt': {'거리': 3.0498591928696674, '단어수': 0}}
행정절차법 일부개정법률안

(유선호의원 대표발의 )

{'1809898.txt': 4.807353198923258}
{'1809898.txt': {'거리': 4.807353198923258, '단어수': 0}}
국군부대의 소말리아 해역 파견연장 동의안

의안

{'1809897.t

---

# 2. Cosine similarity

- euclidean distance와 다르게 summation할때 다르다. 
    - 𝑡∈𝑞∩𝑑

- svm은 l1, l2를 고를 필요가 없다.

- 보통 개발할 때 수집/학습 서버 1대, 서비스 제공 서버 1대 해서 총 2대를 쓴다. (실시간처럼 느끼게 한다.)



In [64]:
# Relevance(Euclidean, Cosine Theta)
# 2. Cosine Theta
searchResult = defaultdict(float)
fp = open("weight.dat", "rb")
queryLength = 0.0
for term, qw in localPostings.items():
    ptr = Lexicon[term]["ptr"]
    fp.seek(ptr)
    for _ in range(Lexicon[term]["df"]):
        _struct = unpack("if", fp.read(4*2))
        dw = _struct[1]
        searchResult[_struct[0]] += (qw*dw)
    queryLength += qw**2
fp.close()

searchResult = {d:angle/sqrt(Documents[d]["length"])\
                            *sqrt(queryLength) # 빼도 무방하지만 빼면 값이 1을 넘어감
               for d, angle in searchResult.items()}

for d, dist in {_[0]:_[1]
                for _ in sorted(searchResult.items(), 
                                key=lambda r:r[1], 
                                reverse=True)}.items():
    print({d:{"각도":dist, "단어수": Documents[d]["ttf"]}})
    print("\n".join(kobill.open(DocumentsList[d]).read().\
                     splitlines()[:3]))
    

{6: {'각도': 7.431129331674336, '단어수': 1803}}
행정절차법 일부개정법률안

(유선호의원 대표발의 )
{4: {'각도': 0.1718470144736527, '단어수': 213}}
고등교육법 일부개정법률안

(안상수의원 대표발의 )
{8: {'각도': 0.04033460946903825, '단어수': 642}}
국군부대의 소말리아 해역 파견연장 동의안

의안
{7: {'각도': 0.0375486104695337, '단어수': 672}}
국군부대의 아랍에미리트(UAE)군 교육훈련 지원 등에 
관한 파견 동의안

{3: {'각도': 0.02805257664723038, '단어수': 893}}
남녀고용평등과 일 ·가정 양립 지원에 관한 법률 

일부개정법률안
{1: {'각도': 0.01849454212767377, '단어수': 883}}
국가공무원법 일부개정법률안

(정의화의원 대표발의 )
{0: {'각도': 0.018212451826866254, '단어수': 883}}
지방공무원법 일부개정법률안

(정의화의원 대표발의 )
{2: {'각도': 0.012825953697034324, '단어수': 1039}}
교육공무원법 일부개정법률안

(정의화의원 대표발의 )
{9: {'각도': 0.012368922287890492, '단어수': 1855}}
결혼중개업의 관리에 관한 법률 일부개정법률안

(한선교의원 대표발의 )
{5: {'각도': 0.0, '단어수': 346}}
하도급거래 공정화에 관한 법률 일부개정법률안

(유선호의원 대표발의 )


In [65]:
Lexicon["현행법"]

{'ptr': 6168, 'df': 5}